<a href="https://colab.research.google.com/github/Nisjain120/Emotion_detection_/blob/main/Emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip -q "/content/archive (18).zip" -d /content/newdata


In [17]:
#mobileNet

In [26]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Define the number of classes in your dataset
num_classes = 7  # Update this value to match your dataset (7 classes based on earlier observations)

# Load pre-trained MobileNetV2 with frozen layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune: Unfreeze some layers in the base model
for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers for fine-tuning
    layer.trainable = True

# Add custom classification layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Replaces Flatten for better performance
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Adjusted num_classes to 7
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Data Generators with resizing and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load datasets
train_data = train_datagen.flow_from_directory(
    '/content/newdata/archive (17)/train',  # Training data path
    target_size=(224, 224),  # Resize to match MobileNetV2
    batch_size=32,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    '/content/newdata/archive (17)/valid',  # Validation data path
    target_size=(224, 224),  # Resize to match MobileNetV2
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    '/content/newdata/archive (17)/test',  # Testing data path
    target_size=(224, 224),  # Resize to match MobileNetV2
    batch_size=32,
    class_mode='categorical'
)

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=15,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Loss: {test_loss:.2f}")


Found 17729 images belonging to 7 classes.
Found 4833 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/15
555/555 ━━━━━━━━━━━━━━━━━━━━ 291s 436ms/step - accuracy: 0.3518 - loss: 1.6579 - val_accuracy: 0.1682 - val_loss: 8.0515 - learning_rate: 0.0010
Epoch 2/15
555/555 ━━━━━━━━━━━━━━━━━━━━ 220s 392ms/step - accuracy: 0.5141 - loss: 1.3072 - val_accuracy: 0.4053 - val_loss: 2.6464 - learning_rate: 0.0010
Epoch 3/15
555/555 ━━━━━━━━━━━━━━━━━━━━ 261s 390ms/step - accuracy: 0.5343 - loss: 1.2466 - val_accuracy: 0.3323 - val_loss: 2.9769 - learning_rate: 0.0010
Epoch 4/15
555/555 ━━━━━━━━━━━━━━━━━━━━ 221s 393ms/step - accuracy: 0.5658 - loss: 1.1720 - val_accuracy: 0.3224 - val_loss: 3.7596 - learning_rate: 0.0010
Epoch 5/15
555/555 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step - accuracy: 0.5819 - loss: 1.1391
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
555/555 ━━━━━━━━━━━━━━━━━━━━ 219s 391ms/step - accuracy: 0.5819 - loss: 1.1391 - val_

In [27]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

# Save the trained model
model.save('emotion_detection_model.h5')  # Save the model for future use

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test Loss: {test_loss:.2f}")

# Function to predict an emotion from a single image
def predict_emotion(image_path, model_path='emotion_detection_model.h5', target_size=(224, 224)):
    """
    Predicts the emotion for a given image.

    Parameters:
    - image_path: Path to the image to predict.
    - model_path: Path to the saved model.
    - target_size: Target size for the image (default is (224, 224)).

    Returns:
    - Predicted class label and probabilities.
    """
    # Check if the file extension is supported
    supported_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    file_extension = os.path.splitext(image_path)[1].lower()
    if file_extension not in supported_extensions:
        raise ValueError(f"Unsupported file extension: {file_extension}. Supported extensions: {supported_extensions}")


225/225 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.6369 - loss: 1.0438
Test Accuracy: 0.64
Test Loss: 1.02


In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Define the prediction function
def predict_emotion(image_path, model_path=r'/content/emotion_detection_model.h5', target_size=(224, 224)):

    # Load the saved model
    model = load_model(r'/content/emotion_detection_model.h5')

    # Load and preprocess the image
    try:
        image = load_img(image_path, target_size=target_size)  # Load image and resize
        image_array = img_to_array(image) / 255.0  # Normalize pixel values
        image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    except Exception as e:
        raise ValueError(f"Error loading or preprocessing image: {e}")

    # Make a prediction
    predictions = model.predict(image_array)
    predicted_class = np.argmax(predictions)  # Get the class with the highest probability

    return predicted_class, predictions

# Example usage
# Upload your image and use its path for prediction
image_path = r'/content/KA.FE1.45.tiff'  # Replace with the actual image path after upload
try:
    predicted_class, predictions = predict_emotion(image_path)
    # Display the results
    class_labels = list(train_data.class_indices.keys())  # Get class labels from the training data
    print(f"Predicted Class: {class_labels[predicted_class]}")
    print(f"Prediction Probabilities: {predictions}")
except ValueError as e:
    print(e)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Class: sad
Prediction Probabilities: [[2.3606446e-02 1.3553229e-04 1.3167349e-01 1.0594956e-03 1.1530323e-01
  7.2770417e-01 5.1772455e-04]]
